In [1]:
import bigframes.pandas as bf

bf.options.bigquery.location = "asia-south2"
bf.options.bigquery.project = "extreme-course-459207-v5"


In [11]:
df = bf.read_gbq("extreme-course-459207-v5.legal_judgement_texts.extracted_judgements") #this variable is set based on the dataset you chose to query

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3553: UserWarning: Reading cached table from 2025-05-18 19:28:44.316159+00:00 to avoid
incompatibilies with previous reads of this table. To read the latest
version, set `use_cache=False` or close the current session with
Session.close() or bigframes.pandas.close_session().
  exec(code_obj, self.user_global_ns, self.user_ns)


In [12]:
# prompt: convert df to pandas

import pandas
df = df.to_pandas()

In [37]:
df.head()

,parties,facts,rulings,legal_citations,id,embedding_text,embedding
0,"[{'name': 'State', 'role': 'Petitioner'} {'na...",Naveen Kumar Dhoulpuria was accused of offense...,The High Court set aside the Order dated 05.12...,['Section 419 IPC' 'Section 420 IPC' 'Section ...,338d177d-039f-41f8-a486-018611b7decf,Facts: Naveen Kumar Dhoulpuria was accused of ...,"[[-0.3664816, -0.10142148, 0.20903732, 0.10813..."
1,"[{'name': 'M S S KUMAR COMPANY', 'role': 'Peti...",The petitioner and respondents entered into a ...,The Court held that the chance for the respond...,"['Arbitration and Conciliation Act, 1996, Sect...",db2e9e40-ddc2-4adf-9fd3-95e2decd19f1,Facts: The petitioner and respondents entered ...,"[[-0.23775125, -0.072568424, 0.26134652, 0.048..."
2,"[{'name': 'Union Public Service Commission', '...","The respondent, Kalpana Ved, applied for a Lim...",The High Court quashed the order passed by the...,"['UOI v Mahendra Singh, 2022 SCC OnLine 909' ...",dbe45cf1-f980-4b39-b9b3-be4c16adbbe7,"Facts: The respondent, Kalpana Ved, applied fo...","[[-0.21794772, -0.10281662, 0.24303815, 0.0594..."
3,"[{'name': 'Sharma Enterprises', 'role': 'Petit...","The petitioner, Sharma Enterprises, a register...",The court dismissed the writ petition with cos...,"['Article 226 of the Constitution of India, 19...",22db70a9-72fc-4e83-91cf-fd32128b13bc,"Facts: The petitioner, Sharma Enterprises, a r...","[[-0.31794262, -0.21112305, 0.25869846, -0.017..."
4,"[{'name': 'M/S INNOVATIVE CRAFTS', 'role': 'Pe...","The petitioner, M/S Innovative Crafts, an expo...",The court set aside the revision order dated 1...,"['Customs Act, 1962 Section 76(1)(b)' 'Customs...",daba1887-6aa8-4236-987b-35a155212464,"Facts: The petitioner, M/S Innovative Crafts, ...","[[-0.20780548, -0.1326801, 0.2849635, 0.057425..."


In [19]:
def format_for_embedding(row):
    print(row)
    facts = str(row["facts"])
    rulings = str(row["rulings"])
    citations = str(row["legal_citations"])
    return f"""Facts: {facts},Rulings: {rulings},citations: {citations}"""

# Example usage with the DataFrame (assuming 'df' is available as per the prompt)
df['embedding_text'] = df.apply(format_for_embedding,axis=1)

parties            [{'name': 'State', 'role': 'Petitioner'}, {'na...
facts              Naveen Kumar Dhoulpuria was accused of offense...
rulings            The High Court set aside the Order dated 05.12...
legal_citations    [Section 419 IPC, Section 420 IPC, Section 468...
id                              338d177d-039f-41f8-a486-018611b7decf
Name: 0, dtype: object
parties            [{'name': 'M S S KUMAR COMPANY', 'role': 'Peti...
facts              The petitioner and respondents entered into a ...
rulings            The Court held that the chance for the respond...
legal_citations    [Arbitration and Conciliation Act, 1996, Secti...
id                              db2e9e40-ddc2-4adf-9fd3-95e2decd19f1
Name: 1, dtype: object
parties            [{'name': 'Union Public Service Commission', '...
facts              The respondent, Kalpana Ved, applied for a Lim...
rulings            The High Court quashed the order passed by the...
legal_citations    [UOI v Mahendra Singh, 2022 SCC OnLine

In [20]:
judgement=df.iloc[0]['embedding_text']

In [21]:
judgement

"Facts: Naveen Kumar Dhoulpuria was accused of offenses under Section 419, 420, 468, 471 & 120B IPC. During the PMT Exam held on 15.05.2011, an invigilator noticed that the person appearing for the exam under Naveen Kumar Dhoulpuria's roll number did not match the photograph on the admit card. Kumar Gaurav was found impersonating Naveen Kumar Dhoulpuria and was handed over to the police. An FIR was registered. The Metropolitan Magistrate framed charges against Naveen Kumar Dhoulpuria, but the Additional Sessions Judge set aside the order and discharged Naveen Kumar Dhoulpuria. The State challenged the order of the Additional Sessions Judge.,Rulings: The High Court set aside the Order dated 05.12.2020 passed by the learned Additional Sessions Judge, thereby reinstating the charges under Section 419, 420, 468, 471 & 120B IPC against the Respondent. The Court held that the fact that someone else was found impersonating the Respondent with a web-downloaded admit card was sufficient to proc

In [23]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("law-ai/InLegalBERT")
text = judgement
encoded_input = tokenizer(text, return_tensors="pt")
model = AutoModel.from_pretrained("law-ai/InLegalBERT")
# output = model(**encoded_input)
# last_hidden_state = output.last_hidden_state


config.json:   0%|          | 0.00/671 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/534M [00:00<?, ?B/s]

In [24]:
output = model(**encoded_input)
last_hidden_state = output.last_hidden_state
last_hidden_state

tensor([[[-0.0748, -0.4057,  0.1965,  ..., -0.5533,  0.2000, -0.1727],
         [-0.1314, -0.5071, -0.0097,  ..., -0.0589,  0.6506,  0.3904],
         [-0.5548,  0.0443, -0.3217,  ..., -0.0197,  0.3939,  0.9136],
         ...,
         [ 0.1151, -0.3708,  0.4145,  ..., -0.2722,  0.8560, -0.3632],
         [ 0.0678, -0.0204,  0.8185,  ..., -0.0059,  0.9055,  0.1812],
         [ 0.0818, -0.0977,  0.3685,  ...,  0.0196,  0.4876,  0.0239]]],
       grad_fn=<NativeLayerNormBackward0>)

In [27]:
import torch

def get_embeddings(text):
    encoded_input = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        max_length=512
    )
    output = model(**encoded_input)
    last_hidden_state = output.last_hidden_state
    return last_hidden_state.mean(dim=1).detach().numpy()


df['embedding'] = df['embedding_text'].apply(get_embeddings)

In [30]:
import numpy as np
import faiss

# Fix: Ensure all embeddings are stacked correctly
embedding_matrix = np.vstack(df['embedding'].values).astype('float32')

# Check the shape
print("Embedding shape:", embedding_matrix.shape)  # Should be (n_samples, embedding_dim)

# Create HNSW index
d = embedding_matrix.shape[1]
index = faiss.IndexHNSWFlat(d, 32)  # 32 = number of neighbors

# Optional tuning
index.hnsw.efConstruction = 200
index.hnsw.efSearch = 50

# Add embeddings
index.add(embedding_matrix)

# Save index
faiss.write_index(index, "judgements_hnsw.index")


Embedding shape: (100, 768)


In [38]:
# Flatten embeddings to 1D arrays
df['embedding'] = df['embedding'].apply(lambda x: x[0] if isinstance(x, (list, np.ndarray)) and len(x) == 1 else x)

from google.cloud import bigquery

def df_to_table(results_df):
    # Define the schema for the BigQuery table
    schema = [
        bigquery.SchemaField("id", bigquery.enums.SqlTypeNames.STRING),
        bigquery.SchemaField("parties", bigquery.enums.SqlTypeNames.RECORD, mode="REPEATED", fields=[
            bigquery.SchemaField("name", bigquery.enums.SqlTypeNames.STRING),
            bigquery.SchemaField("role", bigquery.enums.SqlTypeNames.STRING)
        ]),
        bigquery.SchemaField("facts", bigquery.enums.SqlTypeNames.STRING),
        bigquery.SchemaField("rulings", bigquery.enums.SqlTypeNames.STRING),
        bigquery.SchemaField("legal_citations", bigquery.enums.SqlTypeNames.STRING, mode="REPEATED"),
        bigquery.SchemaField("embedding_text", bigquery.enums.SqlTypeNames.STRING),
        bigquery.SchemaField("embedding", bigquery.enums.SqlTypeNames.FLOAT, mode="REPEATED")
    ]

    job_config = bigquery.LoadJobConfig(
        schema=schema,
        write_disposition="WRITE_TRUNCATE",
    )

    client = bigquery.Client()
    client.load_table_from_dataframe(
        results_df,
        "extreme-course-459207-v5.legal_judgement_texts.extracted_judgements",
        job_config=job_config,
    ).result()  # <- important: wait for the job to finish


In [39]:
df_to_table(df)